In [1]:
import numpy as np
from folktables import ACSDataSource, ACSMobility
from matplotlib import pyplot as plt
import torch
from scipy import stats
from scipy.sparse.linalg import lobpcg
import pandas as pd
from inFairness.distances import MahalanobisDistances, SquaredEuclideanDistance
from inFairness.fairalgo import SenSeI
from inFairness.auditor import SenSeIAuditor
from tqdm.auto import tqdm
from utils import *

/n/home10/colson/.local/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
r = 3
p = 30
n = 200

In [3]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["TX"], download=True)
acs_data_adult = acs_data[acs_data["AGEP"] >= 18]


In [4]:
acs_data_cleaned = acs_data_adult.select_dtypes(include=["float64", "int64"])
acs_data_cleaned = acs_data_cleaned.loc[:, ~(acs_data_cleaned.isna().any())]
acs_data_cleaned = acs_data_cleaned.loc[:, (acs_data_cleaned.var(axis=0) > 0)]
acs_data_cleaned = acs_data_cleaned.sample(frac=1, axis=0)
print(np.shape(np.array(acs_data_cleaned)))


(206826, 219)


In [5]:
print("Generating synthetic data...")
X = clean_data(2 * n, p, acs_data_cleaned)
X_train = X[:n]
X_test = X[n:]

Generating synthetic data...


In [6]:
M, S, y, Astar, Kstar = generate_synthetic_data(n, r, p, X_train)

print(np.shape(X_train), np.shape(X_test))

(200, 30) (200, 30)


In [7]:
A0 = initialization(n, r, p, S, X_train, y)

NameError: name 'r' is not defined